# Table of Contents
* [Setup](#Setup)
	* [estimate cost](#estimate-cost)
	* [load dataset](#load-dataset)
* [Submitting HITs](#Submitting-HITs)
* [Retrieve results](#Retrieve-results)
* [Interact with workers](#Interact-with-workers)
* [Accepting and deleting HITs... careful with these](#Accepting-and-deleting-HITs...-careful-with-these)


In [1]:
%%capture
%load_ext autoreload
%autoreload 2

import pickle
import boto3
import json
import os
from copy import deepcopy
from tqdm import tqdm

from IPython.core.display import HTML

from keysTkingdom import mturk_ai2

from mturk.mturk import MTurk

In [4]:
import os

In [10]:
def unpickle_this(file_name):
    with open(file_name, 'rb') as f:
        results_df = pickle.load(f)
    return results_df

# data

In [5]:
with open('./data/complete_dataset_3_3.pkl', 'rb') as f:
    complete_ds = pickle.load(f)

In [6]:
s3_base_path = 'https://s3-us-west-2.amazonaws.com/ai2-vision-animation-gan/annotation_data/still_frames/'

# Submitting HITs- metric collection

In [40]:
mt_client = MTurk(mturk_ai2.access_key, mturk_ai2.access_secret_key)

Account balance is: $10000.00


In [7]:
static_params = {
    'Title': "sentence word marking",
    'Description': "task description",
    'Keywords': 'task, key, words',
    'frame_height': 1000,
    'Reward': f'{0.01}',
    'AssignmentDurationInSeconds': 3600 * 1,
    'LifetimeInSeconds': 3600 * 24 * 2,
    'MaxAssignments': 1,
}

template_params = {
    'template_file': 'task_template.html', 
    'template_dir': 'hit_templates'
}

In [82]:
sent_len_threshold = 30
current_batch_short_sent = {vid: desc for vid, desc in working_corrected.items() if sent_len_threshold > len(desc.split())}
current_batch_very_long_sent = {vid: desc for vid, desc in working_corrected.items() if sent_len_threshold <= len(desc.split())}

sents = [{'image_id': vid, 'formatted_description': sent} for vid, sent in current_batch_short_sent.items()]
sent = sents[0]

In [37]:
def gen_template_args_from_datapoint(video):
    description = video.description()
    if description[-1] != '.':
        description += '.'
    description = description.replace('\'', '')
    formatted_description = [[word.encode('utf8').decode() for word in sent.split()] for sent in description.split('.')][:-1]
    return {'image_id': video.gid(), 'formatted_description': formatted_description}

In [41]:
test_vid = complete_ds.data[100]

In [83]:
mt_client.preview_hit_interface(template_params, **gen_template_args_from_datapoint(test_vid))

In [50]:
resp = mt_client.create_html_hit(static_params, template_params, **gen_template_args_from_datapoint(test_vid))

# Retreive results

In [55]:
import datetime

all_hits = mt_client.client.list_hits(MaxResults=100)

expire_early = [mt_client.client.update_expiration_for_hit(HITId=v['HITId'], ExpireAt=datetime.datetime(2015, 1, 1)) for v in all_hits['HITs']]

exp = [mt_client.client.update_hit_review_status(HITId=v['HITId'], Revert=False) for v in all_hits['HITs']]

del_resp = [mt_client.client.delete_hit(HITId=v['HITId']) for v in all_hits['HITs']]

In [75]:
def get_all_assignments(mt_client, all_hits=[]):
    assignments = []
    if not all_hits:
        all_hits = mt_client.list_hits(MaxResults=100)['HITs']
        for hit in all_hits:
            assignments.append(mt_client.list_assignments_for_hit(
                HITId=hit['HITId'],
                AssignmentStatuses=['Submitted', 'Approved'],
                MaxResults=10)
            )
    return assignments

In [76]:
assignments = get_all_assignments(mt_client.client)

In [80]:
assignments[0]['Assignments']

[{'AcceptTime': datetime.datetime(2018, 1, 25, 16, 3, 19, tzinfo=tzlocal()),
  'Answer': '<?xml version="1.0" encoding="ASCII"?><QuestionFormAnswers xmlns="http://mechanicalturk.amazonaws.com/AWSMechanicalTurkDataSchemas/2005-10-01/QuestionFormAnswers.xsd"><Answer><QuestionIdentifier>output</QuestionIdentifier><FreeText>{"image_url":"https://s3-us-west-2.amazonaws.com/ai2-vision-animation-gan/annotation_data/user_study/groundtruth/1005.png","selected_image":"pix2pix"}</FreeText></Answer></QuestionFormAnswers>',
  'AssignmentId': '31QTRG6Q2UD1PJ21SAF9I6AC1YOYPU',
  'AssignmentStatus': 'Submitted',
  'AutoApprovalTime': datetime.datetime(2018, 2, 24, 16, 3, 29, tzinfo=tzlocal()),
  'HITId': '3DFYDSXB2X0LZFH4XU8YU5N6LGVUJC',
  'SubmitTime': datetime.datetime(2018, 1, 25, 16, 3, 29, tzinfo=tzlocal()),
  'WorkerId': 'AO235WMFCB1D9'},
 {'AcceptTime': datetime.datetime(2018, 1, 27, 5, 43, 35, tzinfo=tzlocal()),
  'Answer': '<?xml version="1.0" encoding="ASCII"?><QuestionFormAnswers xmlns="htt

In [81]:
for assignment_resp in assignments: 
    assignment = assignment_resp['Assignments']
    if assignment['AssignmentStatus'] == 'Submitted':
        print('Approving Assignment {}'.format(assignment_id))
        client.approve_assignment(
            AssignmentId=assignment_id,
            RequesterFeedback='good',
            OverrideRejection=False,
        )

TypeError: list indices must be integers or slices, not str

In [ ]:
mt_client.client.list_hits(MaxResults=100)

In [ ]:
def delete_all_hits(mt_client):
    all_hits = mt_client.client.list_hits(MaxResults=100)
    expire_early = [mt_client.client.update_expiration_for_hit(HITId=v['HITId'], ExpireAt=datetime(2000, 1, 1)) for v in all_hits['HITs']]

## check results

In [506]:
main_characters =  {"fred",
             "barney",
              "wilma",
              "betty",
              "pebbles",
              "bamm bamm",
              "dino",
              "mr slate",
              "baby puss",
              "hoppy",
              "empty frame",
              "no characters"}

def create_result(assmt):
    result = json.loads(assmt.answers[0][0].fields[0])
    result['h_id'] = assmt.HITId
    result['worker_id'] = assmt.WorkerId
    return result


def all_main(char_set, main_chars=main_characters):
    return not bool(char_set.difference(main_chars))

In [513]:
assignments =[]
for assignment_triple in list(results.values()):
    assignments.extend(assignment_triple)

assignment_results = [create_result(ar) for ar in assignments]
characters_present = [{'h_id': anno['h_id'], 'w_id': anno['worker_id'], 'still_id': anno['stillID'], 'characters': set([ch['label'] for ch in json.loads(anno['characterBoxes'])])} for anno in assignment_results]